In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [134]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import csv


class MyDataset(object):
    '''
    Class to load the dataset
    img_dir = Plots dir
    ground_dir = Ground dir where:
        column 1 = Plot name
        column 2 = Plot ground value
        column 3 = Plot GDD value
    '''
    def __init__(self, img_dir, ground_dir):
        self.img_dir = img_dir
        self.ground_dir = ground_dir
        self.df = self._get_img_df()
        self.ground_arr = self._get_ground_df()
        self.GDD_arr = self._get_GDD_df()
    
    def __len__(self):
        return len(self.df)

    def _get_img_df(self):
        img_names = os.listdir(self.img_dir)
        df = pd.DataFrame(img_names, columns=['Filename'])
        df['Global_ID'] = df.Filename.str[6:-8].astype('int') ###[Global_ID] & [6:-8] to be adjusted
        df['DOY'] = df.Filename.str[-7:-4].astype('int') ###[-7:-4] to be adjusted
        df = pd.pivot_table(df, values = 'Filename', index='Global_ID', columns='DOY', aggfunc='first').reset_index() ###[Global_ID] to be adjusted
        df= df.sort_values('Global_ID')
        return(df)

    def _get_ground_df(self):
        ground_arr =  pd.read_csv(self.ground_dir, usecols=(0,1))
        ground_arr = pd.DataFrame(ground_arr)
        ground_arr = ground_arr.sort_values('Global_ID')
        ground_arr = ground_arr.to_numpy()
        return(ground_arr)

    def _get_GDD_df(self):
        GDD_arr = pd.read_csv(self.ground_dir, usecols=(0,3))
        GDD_arr = pd.DataFrame(GDD_arr)
        GDD_arr = GDD_arr.sort_values('Global_ID')
        GDD_arr = GDD_arr.to_numpy()
        return(GDD_arr)

    def _transform(self, x):
        x = np.moveaxis(x, -3, -2) ###[np.moveaxis(x, -2, -2)] to be adjusted. Dim goals: Plot:Flight:R:G:B
        ### Normalization or rescale did NOT change the results.

        # x = x.astype('float32')

        # layer = Normalization()
        # layer.adapt(x)
        # x = layer(x)
        # x = Rescaling(1.0 / 255)(x)
        # input_shape = x.shape[:2]
        # inputs = keras.Input(input_shape[0],input_shape[1])
        # x = Rescaling(1.0 / 255)(inputs)  # Rescale inputs
        # x /= 255.0

        # min = x.min()
        # max = x.max()
        # x -= min
        # x /= (max - min)
        
        return(x)

    def __getitem__(self,imgs_stacked):
        alist = [] 
        for column in tqdm(self.df.columns[1:]): 
            print(f"Column name:",column)
            imgs_stack = np.stack([cv2.imread(os.path.join(self.img_dir, img)) for img in self.df[column]])
            #imgs_stack = np.stack([cv2.cvtColor(cv2.imread(os.path.join(self.img_dir, img)), cv2.COLOR_BGR2HSV) for img in self.df[column]]) ### Want to change the color space? [cv2.COLOR_BGR2HSV]

            imgs_stack = self._transform(imgs_stack)
            alist.append([imgs_stack])
        imgs_stacked = np.squeeze(np.stack(alist, axis=2))                                                                                           

        return (imgs_stacked) 
        #display(arr)

In [135]:
#%cd /content/drive/MyDrive/Soybean_maturity_Trevisan/
main_dir = '/content/drive/MyDrive/UAS_Beans/Beans_Maturity/2020/SVREC_Mat'
img_dir = os.path.join(main_dir,'e._ClipPlots_BN/P2')

ground_dir = os.path.join(main_dir,'d._Ground_notes', '2020_SVREC_BN_ground2.csv')


In [7]:
dataset_arr1 =  MyDataset(img_dir, ground_dir)[object]
len(dataset_arr1)

  0%|          | 0/6 [00:00<?, ?it/s]

Column name: 70
Column name: 77
Column name: 83
Column name: 90
Column name: 98
Column name: 104


284

In [9]:
dataset_arr2 =  MyDataset(img_dir, ground_dir)[object]
len(dataset_arr2)

  0%|          | 0/6 [00:00<?, ?it/s]

Column name: 58
Column name: 65
Column name: 71
Column name: 78
Column name: 86
Column name: 92


300

In [10]:
dataset_arr2.shape

(300, 6, 256, 64, 3)

In [54]:
image = dataset_arr2[111,5]
image.shape
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (image.min(), image.max()))

Min: 0.000, Max: 255.000


In [136]:
dataset_df =  MyDataset(img_dir, ground_dir).df
dataset_df
#dataset_df.to_csv('C:\\temp_maturity\\UMNSoybeans_Planting_Season_2018\\df.csv')

DOY,Global_ID,58,65,71,78,86,92
0,20031001,SVREC_20031001_058.png,SVREC_20031001_065.png,SVREC_20031001_071.png,SVREC_20031001_078.png,SVREC_20031001_086.png,SVREC_20031001_092.png
1,20031002,SVREC_20031002_058.png,SVREC_20031002_065.png,SVREC_20031002_071.png,SVREC_20031002_078.png,SVREC_20031002_086.png,SVREC_20031002_092.png
2,20031003,SVREC_20031003_058.png,SVREC_20031003_065.png,SVREC_20031003_071.png,SVREC_20031003_078.png,SVREC_20031003_086.png,SVREC_20031003_092.png
3,20031004,SVREC_20031004_058.png,SVREC_20031004_065.png,SVREC_20031004_071.png,SVREC_20031004_078.png,SVREC_20031004_086.png,SVREC_20031004_092.png
4,20031005,SVREC_20031005_058.png,SVREC_20031005_065.png,SVREC_20031005_071.png,SVREC_20031005_078.png,SVREC_20031005_086.png,SVREC_20031005_092.png
...,...,...,...,...,...,...,...
295,20041176,SVREC_20041176_058.png,SVREC_20041176_065.png,SVREC_20041176_071.png,SVREC_20041176_078.png,SVREC_20041176_086.png,SVREC_20041176_092.png
296,20041177,SVREC_20041177_058.png,SVREC_20041177_065.png,SVREC_20041177_071.png,SVREC_20041177_078.png,SVREC_20041177_086.png,SVREC_20041177_092.png
297,20041178,SVREC_20041178_058.png,SVREC_20041178_065.png,SVREC_20041178_071.png,SVREC_20041178_078.png,SVREC_20041178_086.png,SVREC_20041178_092.png
298,20041179,SVREC_20041179_058.png,SVREC_20041179_065.png,SVREC_20041179_071.png,SVREC_20041179_078.png,SVREC_20041179_086.png,SVREC_20041179_092.png


In [ ]:
dataset_ground1 =  MyDataset(img_dir, ground_dir).ground_arr
dataset_ground1

In [ ]:
dataset_GDD1 =  MyDataset(img_dir, ground_dir).GDD_arr
dataset_GDD1

In [145]:
dataset_ground2=  MyDataset(img_dir, ground_dir).ground_arr
dataset_ground2

array([[20031001,       50],
       [20031002,       48],
       [20031003,       49],
       [20031004,       52],
       [20031005,       51],
       [20031006,       51],
       [20031007,       51],
       [20031008,       49],
       [20031009,       50],
       [20031010,       50],
       [20031011,       51],
       [20031012,       51],
       [20031013,       50],
       [20031014,       54],
       [20031015,       49],
       [20031016,       50],
       [20031017,       52],
       [20031018,       49],
       [20031019,       51],
       [20031020,       51],
       [20031021,       50],
       [20031022,       51],
       [20031023,       49],
       [20031024,       52],
       [20031025,       53],
       [20031026,       50],
       [20031027,       52],
       [20031028,       50],
       [20031029,       51],
       [20031030,       52],
       [20031031,       50],
       [20031032,       49],
       [20031033,       50],
       [20031034,       52],
       [200310

In [146]:
dataset_GDD2 =  MyDataset(img_dir, ground_dir).GDD_arr
dataset_GDD2

array([[2.00310010e+07, 2.19638554e+01],
       [2.00310020e+07, 2.22407407e+01],
       [2.00310030e+07, 2.21146342e+01],
       [2.00310040e+07, 2.17000000e+01],
       [2.00310050e+07, 2.18964286e+01],
       [2.00310060e+07, 2.18964286e+01],
       [2.00310070e+07, 2.18964286e+01],
       [2.00310080e+07, 2.21146342e+01],
       [2.00310090e+07, 2.19638554e+01],
       [2.00310100e+07, 2.19638554e+01],
       [2.00310110e+07, 2.18964286e+01],
       [2.00310120e+07, 2.18964286e+01],
       [2.00310130e+07, 2.19638554e+01],
       [2.00310140e+07, 2.13850575e+01],
       [2.00310150e+07, 2.21146342e+01],
       [2.00310160e+07, 2.19638554e+01],
       [2.00310170e+07, 2.17000000e+01],
       [2.00310180e+07, 2.21146342e+01],
       [2.00310190e+07, 2.18964286e+01],
       [2.00310200e+07, 2.18964286e+01],
       [2.00310210e+07, 2.19638554e+01],
       [2.00310220e+07, 2.18964286e+01],
       [2.00310230e+07, 2.21146342e+01],
       [2.00310240e+07, 2.17000000e+01],
       [2.003102

In [20]:
dataset_arr=np.concatenate((dataset_arr1, dataset_arr2),axis=0)

In [21]:
dataset_arr.shape

(584, 6, 256, 64, 3)

In [22]:
dataset_ground=np.concatenate((dataset_ground1, dataset_ground2),axis=0)

In [139]:
dataset_GDD=np.concatenate((dataset_GDD1, dataset_GDD2),axis=0)

In [23]:
dataset_ground.shape

(584, 2)

In [140]:
dataset_GDD.shape

(584, 2)

In [142]:
dataset_GDD

array([[2.00110010e+07, 2.09979167e+01],
       [2.00110020e+07, 2.09979167e+01],
       [2.00110030e+07, 2.09979167e+01],
       ...,
       [2.00411780e+07, 2.22407407e+01],
       [2.00411790e+07, 2.21146342e+01],
       [2.00411800e+07, 2.22407407e+01]])

In [ ]:
# Change the current working directory
#os.chdir('H:\\My Drive\\UMNSoybeans_Planting_Season_2018\\e._npy_data')
os.chdir('C:\\Users\\leoag\\Michigan State University\\MSU Dry Bean Breeding Lab - General\\UAS_Beans\\2020\\SVREC_Mat\\f._numpy_data')
np.save('2020_SVREC_6fly_BN_512_128.npy', dataset_arr)  

In [ ]:
# Change the current working directory
os.chdir('C:\\Users\\leoag\\Michigan State University\\MSU Dry Bean Breeding Lab - General\\UAS_Beans\\2020\\SVREC_Mat\\f._numpy_data')
np.save('2020_SVREC_ground_6fly_BN.npy', dataset_ground)  

In [143]:
# Change the current working directory
os.chdir('/content/drive/MyDrive/UAS_Beans/Beans_Maturity/2020/SVREC_Mat/f._numpy_data')
np.save('2020_SVREC_GDD_6fly_BN.npy', dataset_GDD) 